  # Import Libraries

In [2]:
from typing import Dict, List, Optional, Tuple
from pathlib import Path

from itertools import zip_longest, accumulate, filterfalse
from collections import Counter

import re
import unicodedata

import csv
import pickle

# define custom types
Conllu = List[List[str]]
LabelConlluDict = Dict[str, Conllu]
DocuConlluDict = Dict[str, LabelConlluDict]


  # Define some pathes

In [3]:
file_ar1 = "../outputs/conllu/stanza_ar.conllu"
file_ar2 = "../outputs/conllu/udp_ar.conllu"

file_en1 = "../outputs/conllu/stanza_en.conllu"
file_en2 = "../outputs/conllu/udp_en.conllu"

results_dir = "../outputs/conllu_compare/"


  # Define some functions

In [4]:
def read_text(file):
    with open(file, "r") as f:
        conllu = f.read()
    return conllu



In [5]:
def read_conllu(conllu_file):
    """
    Read a CoNLL-U formatted file and return a dictionary of sentences
    keyed by label IDs and label texts.

    Args:
        conllu_file (str): The file path of the CoNLL-U formatted file.

    Returns:
        dict: A dictionary of sentences keyed by sentence ID and text reference.
    """

    # Define regular expressions to extract sentence ID and text.
    sent_id_regx = re.compile(r"(?<=# sent_id = )\w+")
    text_regex = re.compile(r"(?<=# text = ).+")
    conllu_filter = re.compile(r"#.+\n")

    # Define regular expressions to fix spaces issues in text.
    remove_extra_spaces = re.compile(r'\s\s+')
    add_space_b4r_openbracket = re.compile(r'(?<=\w)\(')
    add_space_b4r_doublequote = re.compile(r'(?<=\w)"')

    conllu = read_text(conllu_file)  # read CoNLL-U file

    # Extract sentence IDs and text from the CoNLL-U formatted file.
    sent_ids = sent_id_regx.findall(conllu)
    texts = text_regex.findall(conllu)
    # Remove comments from the CoNLL-U formatted string.
    sents_conllu = conllu_filter.sub(repl="", string=conllu)

    # Split the CoNLL-U formatted string into sentences and tokens.
    sents_conllu = map(lambda x: x.split("\n"), sents_conllu.split("\n\n"))
    sents_conllu = [[tc.split("\t")[:8] for tc in sc] for sc in sents_conllu]

    # Fix issues with extra spaces in text.
    texts = [remove_extra_spaces.sub(' ', txt) for txt in texts]
    texts = [add_space_b4r_openbracket.sub(' (', txt) for txt in texts]
    texts = [add_space_b4r_doublequote.sub(' "', txt) for txt in texts]
    texts = [unicodedata.normalize("NFKD", txt) for txt in texts]  # /xa0

    # Create a dictionary of sentences keyed by sentence ID and text reference.
    return {
        idx: {
            txt: conllu
        }
        for idx, txt, conllu in zip(sent_ids, texts, sents_conllu)
    }



In [6]:
def extract_conllu_diffs(
    conllu_doc_base: DocuConlluDict, conllu_docu_2compr: DocuConlluDict
) -> List[Tuple[List[str], List[str], List[Conllu]]]:
    """
    Compare two CoNLL-U dictionaries and extract the differences.
    """
    diff_conllu: List[Conllu] = []  # List Conllu with differences
    diff_idx: List[str] = []  # List of labels idx that has different conllus
    diff_text: List[str] = []  # List of labels text that has different conllus

    # Loop over the base CoNLL-U
    for label_idx, label_conllu_base in conllu_doc_base.items():
        for label_text_base, conllu_base in label_conllu_base.items():
            # Get the corresponding CoNLL-U dict {label_text: CoNLLU-U} from
            # the second dictionary to compare with.
            label_conllu_2compr = conllu_docu_2compr.get(label_idx, None)
            if label_conllu_2compr is None:
                # the second dictionary does not have this label
                # TODO: either raise an error or record it as a difference
                pass
            else:
                # get the corresponding CoNLL-U list of list of CoNLL-U fields
                conllu_2compr = label_conllu_2compr.get(label_text_base)
                # Combine the corresponding CoNLL-U fields. If Both CoNLL-Us do
                # not have the same lenghth fill the empty fields with '<none>'
                # [
                #   [(base_token1_idx, 2compr_token1_idx), ...
                #     (base_token1_UPOS, 2compr_token1_UPOS), ...],
                #   [(base_token2_idx, 2compr_token2_idx), ...
                #     (base_token2_UPOS, 2compr_token2_UPOS), ...],
                #   [ (...), (....), ...]
                # ]
                conllu_fields_combined = [
                    list(zip_longest(c1, c2, fillvalue="<none>"))
                    for c1, c2 in zip_longest(
                        conllu_base, conllu_2compr, fillvalue=["<none>"])
                ]
                # Check if there are any differences
                ndiff = all(f1 == f2 for field_tuple in conllu_fields_combined
                            for f1, f2 in field_tuple)
                if not ndiff:
                    combined_fields = []  # type: Conllu # list of list of str
                    # Record the differences between the corresponding
                    # token fields
                    # For each field in conllu if the base conllu field is the
                    # same as the conllu-to-compare field record the base
                    # field. Otherwise, record the differences in the following
                    # format: f'{base-field}[{field_to_compare}]'
                    for field_tuple in conllu_fields_combined:
                        combined_fields.append([
                            f"{field_base}[{field_2compr}]"
                            if field_base != field_2compr else field_base
                            for field_base, field_2compr in field_tuple
                        ])
                    diff_conllu.append(combined_fields)
                    diff_idx.append(label_idx)
                    diff_text.append(label_text_base)

    # Return a list of
    # the labels indeces, labels text, and differing fields
    return diff_idx, diff_text, diff_conllu



In [18]:
def write_conllu_diffs_to_csv(file_path: str, diff_idx: List[str],
                              diff_text: List[str],
                              diff_conllu: List[Conllu]) -> None:
    """
    Write the CoNLL-U formatted sentences from diff_idx, diff_text, and
    diff_conllu to a CSV file.

    Args:
        file_path (str): The file path to write the CSV file to.
        diff_idx (List[str]): A list of labels IDs.
        diff_text (List[str]): A list of labels texts
        diff_conllu (List[List[List[str]]]): A list of CoNLL-U formatted fields.

    """
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        match_diff = re.compile(r'\[.+\]')
        for idx, text, conllu in zip(diff_idx, diff_text, diff_conllu):
            # exclude feats field from printing. If the differences are in
            # feats only do not print the conllu.
            print_flag = False
            for fields in conllu:
                # some fields [ID, Form, Lemma, POS, XPOS, head, udlabel]
                fields_list = fields[:5] + fields[6:8]
                if any(bool(match_diff.search(f)) for f in fields_list):
                    print_flag = True
                    break

            # found differences in any of [ID, Form, Lemma, POS, XPOS, head,
            # udlabel]
            if print_flag:
                writer.writerow([f'# {idx}'])
                writer.writerow([f'# {text}'])
                for fields in conllu:
                    fields_list = fields[:5] + fields[6:8]
                    writer.writerow(fields_list)
                writer.writerow([])


def read_diff_report_comments(path: str):
    msg = 'Number of issues do not match with the number of tools in comments'
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        _ = next(reader)
        all_comments = []
        for row in reader:
            # Check if all cells in the first three columns are not empty
            if all(row[:3]):
                # Process the row here
                tools = row[0].split('|')
                issues = row[1].split('|')
                assert len(tools) == len(issues), f'{row[3]}: {msg}'
                all_comments.extend(list(zip(tools, issues)))

    return all_comments


In [8]:


def get_diff_per_field(
    diff_idx: List[str], diff_conllu: List[Conllu]
) -> Tuple[Dict[str, List[str]], Dict[str, List[str]]]:

    match_diff = re.compile(r'\[.+\]')
    diff_idxs = {'split': [], 'upos': [], 'feat': [], 'head': [], 'udp': []}
    diff_values = {'upos': [], 'feat': [], 'head': [], 'udp': []}

    for idx, conllu in zip(diff_idx, diff_conllu):
        # check that there tokens are identcal. No difference in split. If
        # exists, get the corresponding label id and continue to the next
        # record. If not search for other difference.
        _, t_forms, *_ = zip(*conllu)
        split_diff_bool = bool(match_diff.search(" ".join(t_forms)))
        if split_diff_bool:
            diff_idxs['split'].append(idx)
            continue
        for fields in conllu:
            _, _, _, t_upos, _, t_feat, t_head, t_udp = fields
            # check if there is differences in UPOS. If exists, there is no
            # need to check differences in Feats filed
            upos_diff_bool = bool(match_diff.search(t_upos))
            if upos_diff_bool:
                diff_idxs['upos'].append(idx)
                diff_values['upos'].append(t_upos)
            else:
                feat_diff_bool = bool(match_diff.search(t_feat))
                if feat_diff_bool:
                    diff_idxs['feat'].append(idx)
                    diff_values['feat'].append(t_feat)

            # check if there is differences in UD
            t_head_diff_bool = bool(match_diff.search(t_head))
            if t_head_diff_bool:
                diff_idxs['head'].append(idx)
                diff_values['head'].append(t_head)

            # check if there is differences in head field
            t_udp_diff_bool = bool(match_diff.search(t_udp))
            if t_udp_diff_bool:
                diff_idxs['udp'].append(idx)
                diff_values['udp'].append(t_udp)

    return diff_idxs, diff_values


def examine_field_diff(
        field_diff: List[str],
        threshold: Optional[float] = None
) -> List[Tuple[str, int, float, float]]:

    diff_counts = Counter(field_diff).most_common()

    diff_names, diff_names_count = zip(*diff_counts)
    diff_names: Tuple[str]
    diff_names_count: Tuple[int]

    total_diff_num = sum(diff_names_count)
    diff_name_percent = [dc / total_diff_num for dc in diff_names_count]
    diff_name_percent_cumsum = accumulate(diff_name_percent)
    diff_data = list(
        zip(diff_names, diff_names_count, diff_name_percent,
            diff_name_percent_cumsum))

    if threshold is not None:
        diff_data = list(filterfalse(lambda x: x[-1] > threshold, diff_data))

    return diff_data



In [9]:
def print_diff_report(diff_data: List[Tuple[str, int, float, float]]):
    values, counts, percnts, percent_cumsums = zip(*diff_data)
    v_w = max([len(v) for v in values]) + 2

    header = ['Diff Value', 'Count', '%', "% CumSum"]
    hw = [max(v_w, 11), 9, 9, 9]
    for h, w in zip(header, hw):
        print(f'{h:{w}}', end='')
    print()

    for v, c, p, pcm in diff_data:
        print(f'{v:{hw[0]}}{c:<{hw[1]}}{p:<{hw[2]}.2%}{pcm:<{hw[3]}.2%}')
    print()



  # Compare between Stanza and UDpipe CoNLL-U Outputs

In [19]:
# Extract difference between two CoNLL-U files
diff_idxs_report = []
diff_values_report = []
for file_1, file_2 in [(file_ar1, file_ar2), (file_en1, file_en2)]:
    # Read files
    conllu_dict1 = read_conllu(file_1)  # stanza
    conllu_dict2 = read_conllu(file_2)  # udpipe

    # get differences
    label_ids, label_txts, conllus = extract_conllu_diffs(
        conllu_dict1, conllu_dict2)

    # Count differences per field
    diff_idxs, diff_values = get_diff_per_field(label_ids, conllus)
    diff_idxs_report.append(diff_idxs)
    diff_values_report.append(diff_values)

    # write differences int csv
    f1_name = Path(file_1).stem
    f2_name = Path(file_2).stem
    file_name = f'{results_dir}{f1_name}_vs_{f2_name}'
    with open(f'{file_name}.pkl', "wb") as f:
        pickle.dump([label_ids, label_txts, conllus], f)

    # write csv
    lang = f1_name.split('_')[-1]
    file_name = f'{results_dir}split_diff_conllu_{lang}.csv'
    write_conllu_diffs_to_csv(file_path=file_name,
                              diff_idx=label_ids,
                              diff_text=label_txts,
                              diff_conllu=conllus)

    # print difference percentege
    diff_prnt = len(label_ids) / len(conllu_dict1)
    print(f'Difference between {f1_name} and {f2_name}:')
    print(f'  - Total Percentage:       {diff_prnt:.2%}')
    print(f'  - Total length:           {len(label_ids)}')
    print()


Difference between stanza_ar and udp_ar:
  - Total Percentage:       80.87%
  - Total length:           6181

Difference between stanza_en and udp_en:
  - Total Percentage:       72.67%
  - Total length:           7819



In [10]:
for i, lang in enumerate(['Arabic', 'English']):
    split_diff_num = len(diff_idxs_report[i]['split'])
    split_diff_percent = split_diff_num / len(label_ids)
    print(f'Differences in token split in {lang}: {split_diff_num}', end=" ")
    print(f' which represents {split_diff_percent: .2%} of labels.')


Differences in token split in Arabic: 1548  which represents  19.80% of labels.
Differences in token split in English: 493  which represents  6.31% of labels.


In [11]:
print('Differences count in UPOS')
for i, lang in enumerate(['Arabic', 'English']):
    print(f'in {lang}:')
    diff_data = examine_field_diff(diff_values_report[i]['upos'], 0.99)
    print_diff_report(diff_data)


Differences count in UPOS
in Arabic:
Diff Value   Count    %        % CumSum 
ADJ[NOUN]    1377     38.44%   38.44%   
X[NOUN]      1354     37.80%   76.24%   
NOUN[X]      375      10.47%   86.71%   
ADJ[X]       169      4.72%    91.43%   
NOUN[ADJ]    98       2.74%    94.17%   
X[ADJ]       65       1.81%    95.98%   
VERB[NOUN]   45       1.26%    97.24%   
VERB[X]      26       0.73%    97.96%   
X[ADP]       12       0.34%    98.30%   
NOUN[CCONJ]  5        0.14%    98.44%   
X[CCONJ]     5        0.14%    98.58%   
ADP[X]       4        0.11%    98.69%   
SCONJ[X]     4        0.11%    98.80%   
NOUN[VERB]   4        0.11%    98.91%   
PUNCT[X]     3        0.08%    98.99%   

in English:
Diff Value    Count    %        % CumSum 
NOUN[PROPN]   4226     63.60%   63.60%   
PROPN[NOUN]   833      12.54%   76.13%   
PROPN[ADJ]    253      3.81%    79.94%   
VERB[NOUN]    206      3.10%    83.04%   
ADJ[PROPN]    174      2.62%    85.66%   
PROPN[X]      151      2.27%    87.93%   


In [12]:
print('Differences count in feat')
for i, lang in enumerate(['Arabic', 'English']):
    print(f'in {lang}:')
    diff_data = examine_field_diff(diff_values_report[i]['feat'], 0.99)
    print_diff_report(diff_data)


Differences count in feat
in Arabic:
Diff Value                                                                                                                                                 Count    %        % CumSum 
_[Foreign=Yes]                                                                                                                                             731      39.30%   39.30%   
Case=Nom|Definite=Ind|Number=Sing[Case=Nom|Definite=Cons|Number=Sing]                                                                                      186      10.00%   49.30%   
Foreign=Yes[_]                                                                                                                                             170      9.14%    58.44%   
Case=Nom|Definite=Ind|Number=Sing[Case=Gen|Definite=Ind|Number=Sing]                                                                                       49       2.63%    61.08%   
Case=Gen|Definite=Def|Number=Sing[Case=Gen|Defin

In [13]:
print('Differences count in head')
for i, lang in enumerate(['Arabic', 'English']):
    print(f'in {lang}:')
    diff_data = examine_field_diff(diff_values_report[i]['head'], 0.99)
    print_diff_report(diff_data)


Differences count in head
in Arabic:
Diff Value Count    %        % CumSum 
2[1]       688      21.12%   21.12%   
1[2]       374      11.48%   32.61%   
0[1]       267      8.20%    40.80%   
2[0]       211      6.48%    47.28%   
1[4]       143      4.39%    51.67%   
4[1]       113      3.47%    55.14%   
5[1]       109      3.35%    58.49%   
5[4]       98       3.01%    61.50%   
3[1]       93       2.86%    64.35%   
3[2]       85       2.61%    66.96%   
2[3]       84       2.58%    69.54%   
1[3]       67       2.06%    71.60%   
3[0]       64       1.96%    73.56%   
4[5]       64       1.96%    75.53%   
6[1]       57       1.75%    77.28%   
4[3]       51       1.57%    78.85%   
0[2]       44       1.35%    80.20%   
4[2]       43       1.32%    81.52%   
4[6]       42       1.29%    82.81%   
2[4]       41       1.26%    84.07%   
6[4]       41       1.26%    85.32%   
1[5]       35       1.07%    86.40%   
1[0]       33       1.01%    87.41%   
1[6]       32       0.98%  

In [14]:
print('Differences count in UD')
for i, lang in enumerate(['Arabic', 'English']):
    print(f'in {lang}:')
    diff_data = examine_field_diff(diff_values_report[i]['udp'], 0.99)
    print_diff_report(diff_data)


Differences count in UD
in Arabic:
Diff Value            Count    %        % CumSum 
amod[nmod]            426      20.06%   20.06%   
root[nmod]            270      12.71%   32.77%   
nmod[root]            230      10.83%   43.60%   
nmod[amod]            159      7.49%    51.08%   
nmod[flat:foreign]    135      6.36%    57.44%   
dep[appos]            82       3.86%    61.30%   
obl[nmod]             79       3.72%    65.02%   
nsubj[root]           78       3.67%    68.69%   
dep[nmod]             71       3.34%    72.03%   
punct[case]           65       3.06%    75.09%   
nmod[dep]             50       2.35%    77.45%   
flat:foreign[nmod]    47       2.21%    79.66%   
obj[nmod]             27       1.27%    80.93%   
obl:arg[nmod]         25       1.18%    82.11%   
dep[nummod]           21       0.99%    83.10%   
flat:foreign[dep]     21       0.99%    84.09%   
dep[conj]             18       0.85%    84.93%   
root[amod]            17       0.80%    85.73%   
nsubj[nmod]    

 # Export Differences data

 ## Ckeck differences in split

In [15]:
langs = ['ar', 'en']
lang = 'en'
lang_dict_path = list(Path(results_dir).glob(f'*{lang}.pkl'))[0]

with open(lang_dict_path, 'rb') as f:
    label_ids, label_txts, conllus = pickle.load(f)

split_label_ids = diff_idxs_report[langs.index(lang)]['split']
idx = [label_ids.index(li) for li in split_label_ids]
split_label_txts = [label_txts[i] for i in idx]
split_conllus = [conllus[i] for i in idx]

# write
file_name = f'{results_dir}split_diff_conllu_{lang}.csv'
write_conllu_diffs_to_csv(file_path=file_name,
                          diff_idx=split_label_ids,
                          diff_text=split_label_txts,
                          diff_conllu=split_conllus)


In [17]:
from pprint import pprint

lang = 'ar'
file_name = f'{results_dir}split_diff_conllu_{lang}_commented.csv'
comments = Counter(read_diff_report_comments(file_name)).most_common()
comments_percent = len(comments) / split_diff_num
print(f'Percentage of commints on split difference: {comments_percent:.2%}')
print('comment on split differences')
pprint(comments)


Percentage of commints on split difference: 3.04%
comment on split differences
[(('UDP', 'SplitDefinite'), 55),
 (('UDP', 'MissDetectionOfConnectedPreposition'), 28),
 (('UDP', 'MissDetectionOfConnectedAnd'), 18),
 (('Stanza', 'SplitOfEngWord'), 14),
 (('Stanza', 'MissDetectionOfConnectedPron'), 11),
 (('UDP', 'WrongDetectionOfConnectedPron'), 8),
 (('UDP', 'SplitOfEngWord'), 7),
 (('Stanza', 'MissDetectionOfConnectedPreposition'), 6),
 (('Stanza', 'WrongDetectionOfConnectedAnd'), 4),
 (('None', 'WrongTranslation'), 3),
 (('Stanza', 'WrongDetectionOfConnectedPreposition'), 2),
 (('UDP', 'WrongDetectionOfConnectedPreposition'), 1),
 (('Stanza', 'WrongDetectionofConnectedPreposition'), 1),
 (('Stanza', 'WrongDetectionofConnectedPron'), 1),
 (('UDP', 'WrongDetectionOfConnectedAnd'), 1)]


 ## Notes of split difference

 ### Arabic
 **Stanza detects connected conjunction while UDP detceds connected pronouns.**

 Stanza, unlike UDP, splits the connected Arabic conjunction
 "و", but this comes with high number of positive falses, where the "و" was
 splitted while it is one of the form's letters.

 The same concept with the connected pronouns. UDP has detect the connected
 pronouns but this come with the price of positive error.

 **Stanza detects connected prepositions** unlike UDP, with the price of high
 positive error.

 **UDP split definite words**


 ### English
 The differences are due ti the word/token diffentions when it comes to using
 abbreviations, where dashes, barckets and numbers are used.

 Also, Stanza tend
 to correctly split the 'Apostrophe S'